# Compare Abundance with Mayer
In this notebook we separate Mayer's data into quartiles based on abundances. We then see the proportion of proteins we identify in each quartile.

In [1]:
import pandas as pd
import requests
import os.path
import os
from os import path
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
from matplotlib_venn import venn3

### Access Data
Our data is imported using the longitudinalCLL package and stored using <code>prot</code>.

In [2]:
import longitudinalCLL
prot = longitudinalCLL.get_proteomic()
prot.load_dataset(version='July_noMBR_FP', subjects = [])
payne_df = prot.data_frame.reset_index()

Dowload and import supplementary table 3, use after imputation sheets (Mayer et al., 2018) https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5795392/bin/supp_RA117.000425_133399_0_supp_15943_4ybsvb.xlsx

Since this paper differentiated between nuclear and cytoplasmic, we have to read in 2 files. We then combine the dataframes from the 2 files and extract the protein IDs.

In [3]:
mayer_file = "data/133399_0_supp_15943_4ybsvb.xlsx"

In [4]:
ne_sheet_name = "NE_after imputation"
#The other is "NE_before imputation"
m_ne_df = pd.read_excel(mayer_file, sheet_name = ne_sheet_name)

In [5]:
mayer_cyt_sheet_name = "CYT_after imputation"
m_cyt_df = pd.read_excel(mayer_file, sheet_name= mayer_cyt_sheet_name,
                         skiprows = 1) #There is a header saying sup. table s3

In [6]:
frames = [m_ne_df, m_cyt_df]
m_t = pd.concat(frames)

In [7]:
m_t.dropna(subset = ['Protein IDs'], inplace = True)

In [8]:
m_tot = []

In [9]:
for protein in m_t['Protein IDs']:
    temp = protein.split(";")
    m_tot.append(temp[0])

In [10]:
m_tot = set(m_tot)

### Analysis
Here I select for the healthy subjects in the Mayer paper and average their abundances.

In [11]:
m_ne_df['avg'] = m_ne_df[['Bcells_H3_ne', 'Bcells_H2_ne', 'Bcells_H1_ne']].mean(axis=1)

In [12]:
m_cyt_df['avg'] = m_cyt_df[['Bcells_H3_cyt', 'Bcells_H2_cyt', 'Bcells_H1_cyt']].mean(axis=1)

In [13]:
m_ne_df = m_ne_df[['Protein IDs', 'avg']]
m_cyt_df = m_cyt_df[['Protein IDs', 'avg']]

In [14]:
df = pd.concat([m_ne_df, m_cyt_df]).groupby(['Protein IDs'], as_index=False)['avg'].max()
print (df)

        Protein IDs        avg
0            A0AV96  20.734067
1            A0AVT1  28.003433
2            A0FGR8  24.974000
3            A0JLT2  17.599333
4            A0JNW5  19.484333
...             ...        ...
6940         Q9Y6X5  22.944633
6941  Q9Y6X8;Q9H4I2  26.054300
6942         Q9Y6X9  23.374467
6943         Q9Y6Y0  20.895700
6944         Q9Y6Y8  26.015267

[6945 rows x 2 columns]


I now sort the dataframe by abundance so I can split the data into quartiles based on abundance.

In [15]:
df = df.sort_values(by=['avg'], ascending=False)
df.reset_index()

,index,Protein IDs,avg
0,1324,P16402,36.023900
1,2282,P60709,35.957533
2,2382,P62805,35.945500
3,5245,Q99879;Q99877;Q93079;Q5QNW6;P58876;Q99880,35.233700
4,2504,P84243;Q16695;Q6NXT2,34.774700
...,...,...,...
6940,3994,Q86WW8,16.384300
6941,3559,Q68CR1,16.304100
6942,6882,Q9Y617,16.003400
6943,3235,Q16658,15.744533


Here I figure out how many proteins should be in each quartile and then I create a dataframe with each quartile of data.

In [16]:
increment = df.shape[0]/4
increment

1736.25

In [17]:
q1 = increment
q2 = q1 + increment
q3 = q2 + increment

In [18]:
df_1 = df.iloc[:round(q1), :]
df_1

,Protein IDs,avg
1324,P16402,36.023900
2282,P60709,35.957533
2382,P62805,35.945500
5245,Q99879;Q99877;Q93079;Q5QNW6;P58876;Q99880,35.233700
2504,P84243;Q16695;Q6NXT2,34.774700
...,...,...
1456,P22033,25.756000
2951,Q14156;Q9Y2G0,25.754967
366,O43681,25.754700
73,O00194,25.753567


In [19]:
df_2 = df.iloc[round(q1):round(q2), :]
df_2

,Protein IDs,avg
5970,Q9NR45,25.751400
3234,Q16656,25.749633
1128,P09884,25.745800
4996,Q96JY6,25.743633
1905,P46527,25.742600
...,...,...
1731,P35241,23.301400
6407,Q9UHW5,23.300400
6395,Q9UHJ6,23.298600
4625,Q92575,23.297800


In [20]:
df_3 = df.iloc[round(q2):round(q3), :]
df_3

,Protein IDs,avg
3068,Q15025,23.295367
2859,Q13501,23.294767
108,O00443,23.292133
6236,Q9NZW5;REV__Q8N884,23.288933
3429,Q5RI15,23.281000
...,...,...
5270,Q9BQ70,20.980400
6015,Q9NS87,20.979333
1345,P17252;P05129,20.977667
2815,Q13291,20.977233


In [21]:
df_4 = df.iloc[round(q3):, :]
df_4

,Protein IDs,avg
2253,P57076,20.975567
6649,Q9Y291,20.969867
874,P01344,20.969267
6072,Q9NVA1,20.968800
2936,Q14112,20.967500
...,...,...
3994,Q86WW8,16.384300
3559,Q68CR1,16.304100
6882,Q9Y617,16.003400
3235,Q16658,15.744533


For each quartile I parse the protein IDs so they are in the same format as our data. I then find the intersection between their data and ours. 

In [22]:
m_1 = []
for protein in df_1['Protein IDs'] :
    temp = protein.split(";")
    m_1.append(temp[0])
len(set(m_1).intersection(payne_df['Protein ID']))

1368

In [23]:
ratio1 = len(set(m_1).intersection(payne_df['Protein ID']))/df_1.shape[0]

In [24]:
m_2 = []
for protein in df_2['Protein IDs'] :
    temp = protein.split(";")
    m_2.append(temp[0])
len(set(m_2).intersection(payne_df['Protein ID']))

518

In [25]:
ratio2 = len(set(m_2).intersection(payne_df['Protein ID']))/df_2.shape[0]

In [26]:
m_3 = []
for protein in df_3['Protein IDs'] :
    temp = protein.split(";")
    m_3.append(temp[0])
len(set(m_3).intersection(payne_df['Protein ID']))

181

In [27]:
ratio3 = len(set(m_3).intersection(payne_df['Protein ID']))/df_3.shape[0]

In [28]:
m_4 = []
for protein in df_4['Protein IDs'] :
    temp = protein.split(";")
    m_4.append(temp[0])
len(set(m_4).intersection(payne_df['Protein ID']))

103

In [29]:
ratio4 = len(set(m_4).intersection(payne_df['Protein ID']))/df_4.shape[0]

Finally, I create a table for the data.

In [30]:
from tabulate import tabulate

table_data = [{'quartile 1', ratio1},
              {'quartile 2', ratio2},
              {'quartile 3', ratio3},
              {'quartile 4', ratio4}]
head = ['Ratio','Quartile']
print(tabulate(table_data, headers = head, tablefmt='grid'))

+---------------------+--------------------+
| Ratio               | Quartile           |
+=====================+====================+
| quartile 1          | 0.7880184331797235 |
+---------------------+--------------------+
| 0.29838709677419356 | quartile 2         |
+---------------------+--------------------+
| 0.10420264824409903 | quartile 3         |
+---------------------+--------------------+
| 0.05933179723502304 | quartile 4         |
+---------------------+--------------------+
